# Constants, tools and utils

In this chapter we'll cover some system constants and utility functions.

## `⎕A`

`⎕A` is the [uppercase English alphabet](http://help.dyalog.com/latest/index.htm#Language/System%20Functions/a.htm) :

In [2]:
⎕A

ABCDEFGHIJKLMNOPQRSTUVWXYZ

There is no built-in for the lowercase alphabet, but you can get it with the [case convert](http://help.dyalog.com/latest/index.htm#Language/System%20Functions/c.htm) system function, `⎕C`, using a `¯1` left argument:

In [6]:
¯1⎕C⎕A

abcdefghijklmnopqrstuvwxyz

## `⎕D`

`⎕D` has the [digits](help.dyalog.com/latest/index.htm#Language/System%20Functions/d.htm):

In [7]:
⎕D

0123456789

## `⎕NULL`

`⎕NULL` is a [scalar null](help.dyalog.com/latest/index.htm#Language/System%20Functions/d.htm) value. It isn't really used much in APL itself, but you can meet it e.g. when importing spreadsheets where it represents empty cells.  Note that it is _not_ JSON `null`, which is represented as `⊂'null'` to match true and false being `⊂'true'` and `⊂'false'`.  Note also that `⎕NULL` equals itself. These three (`⎕A ⎕D ⎕NULL`) are system _constants_; you can't assign to them. 

## `⎕SH ⎕CMD`

[`⎕CMD`](http://help.dyalog.com/latest/index.htm#Language/System%20Functions/cmd.htm) and [`⎕SH`](http://help.dyalog.com/latest/index.htm#Language/System%20Functions/sh.htm) are exactly identical, but obviously the first feels more natural to Windows users while the second feels more natural to UNIX users. Pressing f1 on them will give you the help appropriate for that OS. As you, of course, can guess, they are used to call the OS command processor:

In [8]:
⎕SH'ls /'

Applications
Library
System
Users
Volumes
bin
cores
dev
etc
home
opt
private
sbin
tmp
usr
var

## `⎕CSV`

`⎕CSV` will import and export [Character Separated Values](http://help.dyalog.com/latest/index.htm#Language/System%20Functions/csv.htm).

In [9]:
⎕CSV '"abc","def",3' 'S'

┌───┬───┬─┐
│abc│def│3│
└───┴───┴─┘

It has a ton of options for almost anything you could want, including import and export directly to and from text files. 

## `⎕DR`

`⎕DR` is [Data Representation](http://help.dyalog.com/latest/index.htm#Language/System%20Functions/dr.htm). Monadically, it will tell you how an array is represented internally, and dyadically, it allows you to convert between data types: 

In [10]:
⎕DR 42

83

Dyalog APL data type codes have two parts, the 1's place and the rest. The 1's place tells you which kind of data it is, the rest tells you how many bits are used to store it, with one exception: pointers are always 326 even on 64 bit systems. The number 42 gave us 83, where 3 means integer and 8 means 8-bit. 

Dyalog APL has single-bit Boolean arrays, so they are type 11 where the rightmost 1 means Boolean, and the leftmost 1 means 1-bit. 

In [11]:
⎕DR 1 0 1 1 1 0

11

Dyadic `⎕DR` lets you convert between types: 

In [12]:
11⎕DR 42

0 0 1 0 1 0 1 0

This takes the memory which was used to represent 42 and interprets it as if it was a Boolean array. 
You can also combine two steps of `⎕DR` into one. A two-element left argument will interpret the right argument as that type, then convert it to the type given by the second element of the left argument. 

## `⎕FMT`

`⎕FMT` is [ForMaT](http://help.dyalog.com/latest/index.htm#Language/System%20Functions/fmt.htm). It is like a beefed up version of `⍕`. `⍕` returns a vector for single line arguments, and a matrix otherwise. `⎕FMT` always returns a matrix. Also, `⍕` treats control characters as normal characters, while `⎕FMT` will resolve them: 

In [22]:
str←⎕←'abc',(⎕UCS 8),'def' ⍝ 8 is backspace
⍴⎕←⍕str                    ⍝ ⍕ treats backspace as any other char
⍴⎕←⎕FMT str                ⍝ ⎕FMT resolves it

abdef

abdef
7

abdef
1 5

You see that the `'c'` really was erased by the backspace.

Dyadic `⎕FMT` gives you access to a whole new language, namely a formatting specification language. We won't go though all the details here (see docs!), but here's a taste: 

In [21]:
'I3,F5.2' ⎕FMT 2 4⍴⍳8

1 2.00 3 4.00
 5 6.00 7 8.00

The formatting string `I3,F5.2` means that each row should first have an integer, then a float which uses five characters in width and has 2 decimals, then this formatting is cycled as much as needed for all the columns (here twice). 

## `⎕JSON`

`⎕JSON` [imports/exports JSON](http://help.dyalog.com/latest/index.htm#Language/System%20Functions/json.htm). It works for both arrays and objects: 

In [23]:
⎕JSON'[[42,null],"hello"]'

┌───────────┬─────┐
│┌──┬──────┐│hello│
││42│┌────┐││ │
││ ││null│││ │
││ │└────┘││ │
│└──┴──────┘│ │
└───────────┴─────┘

In [25]:
⊢ns←⎕JSON'{"abc":42,"de":null,"f":"hello"}'
ns.(abc f)

#.[JSON object]

┌──┬─────┐
│42│hello│
└──┴─────┘

We can also export from APL to JSON:

In [26]:
⎕JSON ('abc' 1 2 3) 4 5

[["abc",1,2,3],4,5]

Just be aware that if you want to convert and APL string to JSON, you need use the left argument to specify whether you want import (0) or export (1). 

You can also tell ⎕JSON that you want your JSON fully white-spaced: 

In [27]:
⎕JSON⍠'Compact'0⊢('abc' 1 2 3)4 5

[
 [
 "abc",
 1,
 2,
 3
 ],
 4,
 5
]

Finally, whilst you can import any JSON object, not every APL namespace can be exported. For example, a namespace with APL functions cannot be converted to JSON. Again, `⎕JSON` has some more advanced options — see the docs. `⎕JSON` is fully compliant with JSON, though, but we do allow some leniency which allows you to create some JavaScript objects which are not valid JSON. For example,

In [28]:
⎕JSON 'hello' (⊂'world') 

["hello",world]

We opted for a generalised system for strings without quotes, rather than special casing `null`. The I-beam that preceded `⎕JSON` did in fact use `⎕NULL`. By using enclosed strings, we can losslessly roundtrip. 

## `⎕MAP`

[`⎕MAP`](http://help.dyalog.com/latest/index.htm#Language/System%20Functions/map.htm) is a function we'll only mention and not demonstrate (again, see docs). It basically allows you to use a file as an array instead of keeping the array in memory. Very useful. 

## `⎕UCS`

This brings us to [Unicode Convert](http://help.dyalog.com/latest/index.htm#Language/System%20Functions/ucs.htm), `⎕UCS`, which in its monadic form flips characters and their Unicode code points: 

In [29]:
⎕UCS 954 945 955 951 956 941 961 945

καλημέρα

The dyadic form takes a left argument specifying an encoding scheme and converts to and from byte values rather than code points: 

In [30]:
'UTF-8' ⎕UCS 206 179 206 181 206 185 206 177 32 207 131 206 191 207 133

γεια σου

## `⎕VFI`

`⎕VFI` is [Verify and Fix Input](http://help.dyalog.com/latest/index.htm#Language/System%20Functions/vfi.htm). It takes a string and returns two lists. It cuts the string into space separated fields. Then it attempts to convert each field to a number. If it succeeds then the corresponding element of the left result list is 1 (else 0) and the corresponding element of the right list is the number (else 0). 

In [31]:
⎕VFI '123 four 42'

┌─────┬────────┐
│1 0 1│123 0 42│
└─────┴────────┘

You can also specify one or more valid field separators as left argument: 

In [32]:
';/'⎕VFI '123 four,42 5/2/4'

┌─────┬─────┐
│0 1 1│0 2 4│
└─────┴─────┘

Here `123 four` were grouped because space is not a separator anymore, and so it is an invalid number. So too with `42 5`. Only `2` and `4` were valid. You can get just the valid numbers with: 

In [33]:
//';/'⎕VFI '123 four,42 5/2/4'

┌───┐
│2 4│
└───┘

## `⎕XML`

`⎕XML` is converts to and from XML, but the corresponding APL format is rather involved. We usually just use ⎕XML to verify that some XML is valid or to normalise whitespace: 

In [34]:
⎕XML⍣2 ⊢ '<xml><document id="001">An introduction to XML</document></xml>'

<xml>
 <document id="001">An introduction to XML</document>
</xml>